# Python网络爬虫 - 股票数据爬取 
#### Author: Arianna.H
#### Date: Oct 12, 2019
  
  ___
Task: 获取东方财富网中当日沪深A股股票名称和交易数据 

original url: http://quote.eastmoney.com/center/gridlist.html#hs_a_board   

idea: urllib.request—>response—>bs4 parser—>get and transform data—>save csv file  

problem: 网页改版后，源代码没有table信息，股票信息不能从html网页直接获取，需利用开发者工具分析网页请求来源（通过js加载）  

In [4]:
pd.set_option('display.max_columns', None)

In [3]:
import pandas as pd
import urllib.request as r
import bs4
from bs4 import BeautifulSoup

* 使用Google Chrome，F12进入Devtool，通过record点击下一页的网页请求，发现数据通过js加载，复制js链接：

沪深A股
url_shsz="http://78.push2.eastmoney.com/api/qt/clist/get?cb=jQuery1124048191705730265766_1570855507672&pn=1&pz=20&po=1&np=1&ut=bd1d9ddb04089700cf9c27f6f7426281&fltt=2&invt=2&fid=f3&fs=m:0+t:6,m:0+t:13,m:0+t:80,m:1+t:2,m:1+t:23&fields=f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f12,f13,f14,f15,f16,f17,f18,f20,f21,f23,f24,f25,f22,f11,f62,f128,f136,f115,f152&_=1570855507749"

上证A股
url_sh="http://78.push2.eastmoney.com/api/qt/clist/get?cb=jQuery1124048191705730265766_1570855507672&pn=1&pz=20&po=1&np=1&ut=bd1d9ddb04089700cf9c27f6f7426281&fltt=2&invt=2&fid=f3&fs=m:1+t:2,m:1+t:23&fields=f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f12,f13,f14,f15,f16,f17,f18,f20,f21,f23,f24,f25,f22,f11,f62,\f128,f136,f115,f152&_=1570855507751"

深证A股
url_sz="http://78.push2.eastmoney.com/api/qt/clist/get?cb=jQuery1124048191705730265766_1570855507672&pn=1&pz=20&po=1&np=1&ut=bd1d9ddb04089700cf9c27f6f7426281&fltt=2&invt=2&fid=f3&fs=m:0+t:6,m:0+t:13,m:0+t:80&fields=f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f12,f13,f14,f15,f16,f17,f18,f20,f21,f23,f24,f25,f22,f11,f62,f128,f136,f115,f152&_=1570855507753"
   
   
* 观察链接得知以下的规律：  
  
pn为page number, pz为page zise； fs=m:0为深证，fs=m:1为上证   
  
  

In [5]:
#通过观察解析后的沪深A股url，得知total size=3823
#修改pz=20为pz=3823，可在一页中获取所有股票数据

url_shsz='http://78.push2.eastmoney.com/api/qt/clist/get?cb=jQuery1124048191705730265766_1570855507672\
&pn=1&pz=3823&po=1&np=1&ut=bd1d9ddb04089700cf9c27f6f7426281&fltt=2&invt=2&fid=f3&fs=m:0+t:6,m:0+t:13,\
m:0+t:80,m:1+t:2,m:1+t:23&fields=f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f12,f13,f14,f15,f16,f17,f18,f20,f21,\
f23,f24,f25,f22,f11,f62,f128,f136,f115,f152&_=1570855507749'

In [6]:
#模拟浏览器访问反爬取沪深A股
#读取第一页数据

headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.87 Safari/537.36'}
request = r.Request(url_shsz, headers=headers)
response = r.urlopen(request)
html = response.read().decode('utf-8','ignore')
soup = BeautifulSoup(html,'html.parser')

In [7]:
print(soup.prettify())

jQuery1124048191705730265766_1570855507672({"rc":0,"rt":6,"svr":182481158,"lt":1,"full":1,"dlmkts":"","data":{"total":5359,"diff":[{"f1":2,"f2":9.34,"f3":20.05,"f4":1.56,"f5":200303,"f6":185665111.16,"f7":7.58,"f8":8.2,"f9":34.11,"f10":2.08,"f11":0.0,"f12":"300407","f13":0,"f14":"凯发电气","f15":9.34,"f16":8.75,"f17":8.75,"f18":7.78,"f20":2971992605,"f21":2281389091,"f22":0.0,"f23":1.67,"f24":15.74,"f25":-1.06,"f62":56217256.0,"f115":28.0,"f128":"-","f140":"-","f141":"-","f136":"-","f152":2},{"f1":2,"f2":15.82,"f3":20.03,"f4":2.64,"f5":1382818,"f6":2135627058.4,"f7":7.66,"f8":22.56,"f9":24.93,"f10":3.46,"f11":0.0,"f12":"301301","f13":0,"f14":"川宁生物","f15":15.82,"f16":14.81,"f17":14.9,"f18":13.18,"f20":35164696000,"f21":9696273488,"f22":0.0,"f23":4.8,"f24":83.1,"f25":88.33,"f62":-85957072.0,"f115":31.46,"f128":"-","f140":"-","f141":"-","f136":"-","f152":2},{"f1":2,"f2":43.6,"f3":20.01,"f4":7.27,"f5":79131,"f6":321287990.0,"f7":18.97,"f8":11.57,"f9":10.15,"f10":2.89,"f11":0.0,"f12":"688253","

In [9]:
import json
#读取json文档，转换为python dictionary格式
jsondata = soup.prettify().split('(')[1].split(')')[0]
dictdata = json.loads(jsondata)
share_info = dictdata['data']['diff']

In [10]:
pd.DataFrame.from_dict(share_info).head()

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f20,f21,f22,f23,f24,f25,f62,f115,f128,f140,f141,f136,f152
0,2,9.34,20.05,1.56,200303,1.856651e+08,7.58,8.20,34.11,2.08,0.0,300407,0,凯发电气,9.34,8.75,8.75,7.78,2971992605,2281389091,0.0,1.67,15.74,-1.06,56217256.0,28.00,-,-,-,-,2
1,2,15.82,20.03,2.64,1382818,2.135627e+09,7.66,22.56,24.93,3.46,0.0,301301,0,川宁生物,15.82,14.81,14.90,13.18,35164696000,9696273488,0.0,4.80,83.10,88.33,-85957072.0,31.46,-,-,-,-,2
2,2,43.60,20.01,7.27,79131,3.212880e+08,18.97,11.57,10.15,2.89,0.0,688253,1,英诺特,43.60,36.71,37.00,36.33,5932251578,2982819052,0.0,2.98,55.27,74.33,16633284.0,22.28,-,-,-,-,2
3,2,25.07,20.01,4.18,64694,1.570134e+08,20.06,26.41,35.76,3.51,0.0,301209,0,联合化学,25.07,20.88,20.88,20.89,2005600000,614215000,0.0,2.85,13.95,0.44,35778515.0,52.35,-,-,-,-,2
4,2,33.77,20.01,5.63,131500,4.223906e+08,13.36,38.85,133.74,4.74,0.0,301258,0,富士莱,33.77,30.01,30.52,28.14,3095695900,1142970978,0.0,1.59,16.89,4.62,34635334.0,46.65,-,-,-,-,2


In [12]:
import numpy as np
#清理数据，读取成表格
share_tb = pd.DataFrame.from_dict(share_info)
share_tb = share_tb.loc[:,['f13', 'f12', 'f14', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f15', 'f16', 'f17', 'f18', 'f10', 'f8', 'f9', 'f23' ]]
share_tb.columns = ['交易所','序号','名称','最新价','涨跌幅','涨跌额','成交量(手)','成交额','振幅','最高','最低','今开','昨收','量比','换手率','市盈率(动态)','市净率']
share_tb['交易所'] = share_tb['交易所'].map(lambda x: np.where(x==1,'SH','SZ'))
share_tb

,交易所,序号,名称,最新价,涨跌幅,涨跌额,成交量(手),成交额,振幅,最高,最低,今开,昨收,量比,换手率,市盈率(动态),市净率
0,SZ,300407,凯发电气,9.34,20.05,1.56,200303,1.856651e+08,7.58,9.34,8.75,8.75,7.78,2.08,8.20,34.11,1.67
1,SZ,301301,川宁生物,15.82,20.03,2.64,1382818,2.135627e+09,7.66,15.82,14.81,14.90,13.18,3.46,22.56,24.93,4.80
2,SH,688253,英诺特,43.60,20.01,7.27,79131,3.212880e+08,18.97,43.60,36.71,37.00,36.33,2.89,11.57,10.15,2.98
3,SZ,301209,联合化学,25.07,20.01,4.18,64694,1.570134e+08,20.06,25.07,20.88,20.88,20.89,3.51,26.41,35.76,2.85
4,SZ,301258,富士莱,33.77,20.01,5.63,131500,4.223906e+08,13.36,33.77,30.01,30.52,28.14,4.74,38.85,133.74,1.59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3818,SZ,300173,福能东方,4.23,0.71,0.03,163291,6.899086e+07,2.38,4.29,4.19,4.25,4.20,0.96,2.22,17.41,3.28
3819,SH,600018,上港集团,5.65,0.71,0.04,371980,2.100188e+08,1.96,5.69,5.58,5.68,5.61,1.58,0.16,8.90,1.07
3820,SZ,002421,达实智能,2.83,0.71,0.02,274942,7.822345e+07,2.49,2.89,2.82,2.87,2.81,1.09,1.38,391.15,1.76
3821,SZ,000702,正虹科技,4.27,0.71,0.03,91140,3.900918e+07,3.07,4.34,4.21,4.27,4.24,0.91,3.42,-43.18,3.53


In [13]:
#储存，encoding='utf_8_sig'防止中文乱码
today = '2019_10_12' 
share_tb.to_csv('share_shsz_'+today+'.csv', encoding='utf_8_sig', index=False)